In [1]:
#insillico hull indi in py


import json
import asyncio
import aiohttp
import math

In [ ]:
PERIOD = 55

In [ ]:
async def get_close_data():
    close_arr = []
    #url = "https://api.pro.coinbase.com/products/BTC-USD/candles?granularity=900"
    url = "https://api.pro.coinbase.com/products/BTC-USD/candles?granularity=300"
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp:
            resp = await resp.json()
            for entry in resp:
                close = entry[4]
                close_arr.append(close)
            return close_arr

In [ ]:
def calculate_wma_from_range(data, index_1, index_2):
    sum = 0
    weight = index_2 - index_1
    for i in range(index_1, index_2):
        close = data[i]
        sum += close * weight
        weight -= 1
    
    n = index_2 - index_1
    denom = ((n*n) + n) / 2
    return sum / denom

In [ ]:
async def get_wma_arr(data, num_bars):
    max_size = len(data) - PERIOD
    wma_arr = []
    for i in range(max_size):
        data_item = calculate_wma_from_range(data, i, i+num_bars)
        wma_arr.append(data_item)
    
    return wma_arr

In [ ]:
async def get_hull():
    data = await get_close_data()
    full = await get_wma_arr(data, PERIOD)
    half = await get_wma_arr(data, int(PERIOD/2))
    hma_data = []
    
    for i, j in zip(full, half):
        hma_data.append(2*j - i)

    hma_arr = await get_wma_arr(hma_data, int(math.sqrt(PERIOD)))
    return hma_arr, data

In [ ]:
hull,data = await get_hull()
print(hull[0])
print(hull[2])

# 0 < 2 means latest price calc is lower, red
# 0 > 2 other way